# OPTaaS Constraints
Constraints allow you to specify relationships between the parameters you want to optimize, or just constrain the values that a parameter can take. 

For example, you may want an optional parameter to always be enabled if another parameter is set to `True`, or you may want an IntParameter to be between 0 and 10, but never 3.

## Numeric constraints
You can use all the familiar numeric and comparison operators.

Constraints are automatically converted to string expressions in a format that can be parsed by OPTaaS.

In [1]:
from mindfoundry.optaas.client.parameter import IntParameter
from mindfoundry.optaas.client.constraint import Constraint

x = IntParameter("x", id="id_x", minimum=0, maximum=20)
y = IntParameter("y", id="id_y", minimum=0, maximum=20)

Constraint(x<y)

#id_x < #id_y

In [2]:
Constraint(x+y == 25)

( #id_x + #id_y ) == 25

In [3]:
Constraint(x**2 < y%2)

( #id_x ** 2 ) < ( #id_y % 2 )

If you don't specify an `id` for a parameter, the object id will be used instead - but it's not very readable!

In [4]:
no_id = IntParameter("no_id", minimum=0, maximum=20)

Constraint(x // y <= no_id)

( #id_x // #id_y ) <= #1812494461248

## Logical operators
Use the binary operators `|` and `&` instead of `or` and `and` (this is because `or` and `and` cannot be overridden).

Enclose the operands in brackets to avoid any errors due to the precedence of the binary operators.

In [5]:
Constraint((x<3) | (x>5) & (y>=7))

( #id_x < 3 ) || ( ( #id_x > 5 ) && ( #id_y >= 7 ) )

## Optional parameters
Use `is_present` and `is_absent` to create constraints based on whether a parameter is present in a configuration.

In [6]:
from mindfoundry.optaas.client.parameter import FloatParameter

z = FloatParameter("z", id="id_z", minimum=0, maximum=1, optional=True)

Note that you can create conditional constraints that only apply when a condition is true (in this case, if `z` is present).

In [7]:
Constraint(when=z.is_present(), then=z >= x/y)

if #id_z is_present then #id_z >= ( #id_x / #id_y )

In [8]:
Constraint(when=(x==0) & (y==0), then=z.is_absent())

if ( #id_x == 0 ) && ( #id_y == 0 ) then #id_z is_absent

## Choices
You can use `is_present` and `is_absent` with parameters that are part of a choice.

In [9]:
from mindfoundry.optaas.client.parameter import ChoiceParameter

choice = ChoiceParameter("xy", [x, y], id="id_choice", default=y)

In [10]:
Constraint(when=x.is_present(), then=z<0.5)

if #id_x is_present then #id_z < 0.5

You can also use `==` and `!=` with choices:

In [11]:
Constraint(when=choice==x, then=z<0.5)

if #id_choice == #id_x then #id_z < 0.5

In [12]:
Constraint(when=z==1, then=choice!=x)

if #id_z == 1 then #id_choice != #id_x

## Categoricals
You can use `==`, `!=`, `is_present` and `is_absent` with categoricals as well.

In [13]:
from mindfoundry.optaas.client.parameter import CategoricalParameter

abc = CategoricalParameter('abc', ['a', 'b', 'c'], id='id_abc')

Constraint(when=abc == "a", then=x == 0)

if #id_abc == 'a' then #id_x == 0

In [14]:
Constraint(when=x > 1, then=abc != "c")

if #id_x > 1 then #id_abc != 'c'

## Use them when creating a Task

In [19]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

task = client.create_task(
    title='My Task With Constraints', 
    parameters=[abc, choice, z],
    constraints=[
        Constraint(when=(x==0) & (y==0), then=z.is_absent()),
        Constraint(when=choice==x, then=z < 0.5),
        Constraint(when=x==1, then=abc != "c")
    ]
)

task.generate_configuration()

{ 'id': '64fe9480-d373-4a76-8a3b-ddf8d126d518',
  'type': 'default',
  'values': {'abc': 'a', 'xy': {'y': 10}, 'z': 0.5}}

All generated configurations will obey the constraints:

In [23]:
for _ in range(5):
    display(task.generate_configuration())

{ 'id': 'c042bf9d-fec7-4835-aa27-1fcf23b891a2',
  'type': 'exploration',
  'values': {'abc': 'b', 'xy': {'y': 3}}}

{ 'id': '3cc16756-9d59-4c82-8210-cf2413b11b5a',
  'type': 'exploration',
  'values': {'abc': 'b', 'xy': {'x': 17}, 'z': 0.006360348500056512}}

{ 'id': 'e018ea8d-d0ed-44b9-b663-077e55f19d65',
  'type': 'exploration',
  'values': {'abc': 'c', 'xy': {'y': 15}}}

{ 'id': '08990e9c-7ae1-4236-84fb-df46edfc9e69',
  'type': 'exploration',
  'values': {'abc': 'b', 'xy': {'x': 17}, 'z': 0.3050585562053444}}

{ 'id': '3531575c-cdf8-4d7e-988b-9fcb98e4ce80',
  'type': 'exploration',
  'values': {'abc': 'b', 'xy': {'y': 1}}}

If a configuration violates a constraint, it will not be accepted:

In [21]:
task.add_user_defined_configuration({'xy': {'x': 1}, 'z': 0.5, 'abc': 'c'})

OPTaaSError: Configuration violates constraint: if #id_choice == #id_x then #id_z < 0.5